# Initialization

In [61]:
import os, sys
import time
import numpy as np
import pandas as pd
import random
from scipy import stats as st
import itertools
import operator

import torch

from tqdm.notebook import trange
from tqdm import tqdm

random_state = np.random.RandomState(2020)

In [62]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb')
%run $helpers_file

# Load spotlight module
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(base_dir, p))
    if module_path not in sys.path:
        sys.path.append(module_path)

# Load Dataset

## Models

The following code defined two pre-trained recommendation model to be explained in this project: lstm and pooling. Because lstm also concerns the order of the initial interaction list, it is not suitable for the genetic algorithm method. Therefore, only pooling model is used for testing and bechmarking purposes.

In [63]:
lstm_model = load_model(model_type='entire')
pooling_model = load_model('pooling')

pretrained_models = {
    'lstm': lstm_model,
    'pooling': pooling_model,
}

## Dataset

In [64]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length)

# Genetic Search

## Test Initialization

In [65]:
test_interaction = test.sequences[test.user_ids == 3][0].copy()
test_interaction = test_interaction[test_interaction != 0]
test_interaction.sort()
test_interaction

array([ 59, 114, 124, 125, 177, 186, 190, 191, 196, 197, 200], dtype=int32)

In [66]:
len_test_interaction = len(test_interaction)
len_test_interaction

11

For a specific instance, this function takes a position and gives the item id in that position of the predicted item list.

In [67]:
def get_position_item(model, test_interaction, position=1):
    prediction = model.predict(test_interaction)
    prediction[test_interaction] = -StaticVars.FLOAT_MAX
    rk_data = st.rankdata(-prediction, method='ordinal')
    index = np.where(rk_data == position)
    return index[0][0]

In [68]:
get_position_item(pooling_model, test_interaction, 2)

167

## Random CF candidate selection

This function randomly generates a set of subsets from a specific array. The number of array to be generated is defined by "sublists_info"

In [69]:
import numpy as np

def generate_random_sublists(original_list, sublists_info):
    result_sublists = []
    rng = np.random.default_rng(seed=2020)  # Seed for reproducibility

    for length, count in sublists_info.items():
        generated_sublists_for_length = set()

        while len(generated_sublists_for_length) < count:
            sublist = tuple(rng.choice(original_list, length, replace=False))
            generated_sublists_for_length.add(sublist)

        result_sublists.extend(np.array(list(sublist)) for sublist in generated_sublists_for_length)

    return result_sublists


## Crossover and Mutation

The following two functions perform crossover and mutation job. The crossover function takes two parents and returns two children. The mutation function takes a parent and returns a child. The number of elements to crossover and mutate is defiend by the probablity parameter. For instance, 0.3 crossover prob means that 30% of the elements will be swapped.

In [70]:
import numpy as np

def crossover(first_list, second_list, p_cross):
    # Find the shorter length among the two lists
    length_first = len(first_list)
    length_second = len(second_list)
    shorter_length = min(length_first, length_second)
    
    # Compute the number of crossover points
    num_crossovers = max(int(shorter_length * p_cross), 1)
    
    # Choose random indices for crossover
    rng = np.random.default_rng(seed=2020)
    crossover_indices_first = rng.choice(shorter_length, num_crossovers, replace=False)
    crossover_indices_second = rng.choice(shorter_length, num_crossovers, replace=False)
    
    # Sort the crossover indices
    crossover_indices_first.sort()
    crossover_indices_second.sort()
    
    # Swap the elements at the crossover indices
    for i in range(num_crossovers):
        index_first = crossover_indices_first[i]
        index_second = crossover_indices_second[i]
        first_list[index_first], second_list[index_second] = second_list[index_second], first_list[index_first]
    
    return first_list, second_list

def mutate_array(org_arr, arr_to_mutate, mutation_probability):
    # Calculate the number of elements to mutate
    num_mutations = max(int(mutation_probability * len(arr_to_mutate)), 1)
    rng = np.random.default_rng(seed=2020)
    # Select the indices to mutate
    indices_to_mutate = rng.choice(range(len(arr_to_mutate)), size=num_mutations, replace=False)
    
    # Mutate the selected elements
    for idx in indices_to_mutate:
        arr_to_mutate[idx] = rng.choice(org_arr)

    return arr_to_mutate


def remove_duplicates(arr):
    _, idx = np.unique(arr, return_index=True)
    return arr[np.sort(idx)]

## Loss Functions

There are three loss functions defined: y-loss, distance, and y-loss for subsets of removing items (supersets of candidate CFs).
- y-loss: How far is the target item away from the k-th item, concerning their prediction scores.
- distance: How many elements are removed from the interaction list

In [71]:
from itertools import combinations, chain
import numpy as np
import scipy.stats as st

class StaticVars:
    FLOAT_MAX = float('inf')

def supersets_of_new_subsets_of_old(new_cf, old_cf):
    diff = np.setdiff1d(old_cf, new_cf)  # Elements that are in old_cf but not in new_cf
    for r in range(1, len(diff) + 1):
        for subset in combinations(diff, r):
            yield np.union1d(new_cf, subset)

def compute_yloss(target_score, kth_score):
    yloss = max(0, target_score / kth_score - 1.0)
    return yloss

def compute_distance(x, y):
    diff = np.setdiff1d(x, y)
    return len(diff)

def compute_loss(old_cf, new_cf, model, target_item, top_k, yloss_cache):
    cache_key = frozenset(new_cf)
    if cache_key in yloss_cache:
        yloss = yloss_cache[cache_key]
    else:
        new_prediction = model.predict(new_cf)
        new_prediction[new_cf] = -StaticVars.FLOAT_MAX
        new_rk_data = st.rankdata(-new_prediction, method='ordinal')

        top_k_index = np.where(new_rk_data == top_k)[0][0]
        yloss = compute_yloss(new_prediction[target_item], new_prediction[top_k_index])
        yloss_cache[cache_key] = yloss
    dis = compute_distance(old_cf, new_cf)

    subset_yloss = 0
    for superset in supersets_of_new_subsets_of_old(new_cf, old_cf):
        cache_key = frozenset(superset)
        if cache_key in yloss_cache:
            # print("Cache hit")
            subset_yloss += yloss_cache[cache_key]
        else:
            subset_prediction = model.predict(superset)
            subset_prediction[superset] = -StaticVars.FLOAT_MAX
            sub_rk_data = st.rankdata(-subset_prediction, method='ordinal')
            sub_top_k_index = np.where(sub_rk_data == top_k)[0][0]
            subset_yloss += compute_yloss(subset_prediction[target_item], subset_prediction[sub_top_k_index])
            yloss_cache[cache_key] = subset_yloss

    return list([yloss, dis, subset_yloss])


# NSGA-II
Apply NSGA-II to the problem of finding the optimal candicates in multi-objective optimization problem. It gives ranks to all candidate CFs based on:
- Non-domination Rank
- Crowding Distance

In [72]:
def dominates(row, candidateRow):
    """Determine if one solution dominates another"""
    return all(r <= cr for r, cr in zip(row, candidateRow)) and any(r < cr for r, cr in zip(row, candidateRow))

def crowding_distance_assignment(front, values):
    distances = [0] * len(values)  # Initialize the distance for every solution as 0
    num_objs = len(values[0])
    
    for m in range(num_objs):
        sorted_front = sorted(front, key=lambda x: values[x][m])

        # Assign infinite distance at boundaries.
        distances[sorted_front[0]] = distances[sorted_front[-1]] = float('inf')

        # Normalize the objective values for distance computation.
        obj_min = values[sorted_front[0]][m]
        obj_max = values[sorted_front[-1]][m]
        denom = obj_max - obj_min if obj_max != obj_min else 1

        for i in range(1, len(sorted_front) - 1):
            distances[sorted_front[i]] += (values[sorted_front[i + 1]][m] - values[sorted_front[i - 1]][m]) / denom

    return distances



def fast_nondominated_sort(values):
    """NSGA-II's fast non-dominated sort"""
    S = [[] for _ in range(len(values))]
    front = [[]]
    n = [0 for _ in range(len(values))]
    rank = [-1 for _ in range(len(values))]
    
    for p in range(len(values)):
        S[p] = []
        n[p] = 0
        for q in range(len(values)):
            if dominates(values[p], values[q]):
                S[p].append(q)
            elif dominates(values[q], values[p]):
                n[p] += 1
        if n[p] == 0:
            rank[p] = 0
            front[0].append(p)
            
    i = 0
    while front[i]:
        nextFront = []
        for p in front[i]:
            for q in S[p]:
                n[q] = n[q] - 1
                if n[q] == 0:
                    rank[q] = i + 1
                    nextFront.append(q)
        i = i + 1
        front.append(nextFront)

    del front[len(front) - 1]
    
    # Initialize crowding distances as zeros
    crowding_distances = [0] * len(values)
    
    for front_solutions in front:
        current_front_distances = crowding_distance_assignment(front_solutions, values)
        for j, solution in enumerate(front_solutions):
            crowding_distances[solution] = current_front_distances[solution]
    
    return rank, crowding_distances


# Pipeline

Given a list of arrays, this funciton randomly paris them in groups of 2.

In [73]:
def generate_random_pairs(list_of_arrays, n):
    # Generate all possible pairs
    random.seed(2020)
    all_pairs = list(itertools.combinations(list_of_arrays, 2))

    # Randomly select n pairs
    random_pairs = random.sample(all_pairs, n)

    return random_pairs

This function execute one generation of genetic algorithm. It takes parameters:
- interaction: the original interaction list
- candidates: the candidate CFs from the last generation
- model: the model to be explained
- target: the target item
- k: the top k items to consider. When target item drops out of top k recommendation list, the algorithm gives a valid explaination
- yloss_cache: a dictionary to store the y-loss values of candidates already explored. This is a dynamic programming strategy to speed up the execution process
- crossover_p: the probability of crossover process
- mutation_p: the probability of mutation process
- budget: the budget set for each explaination

The algorithm runs in the following stages:
- Pair all candidate CFs from the last generation
- Do crossover and mutation for each pair
- Compute losses of both old and new generation candidate CFs
- Check if there is an explaination found
    - If found, return the valid explainations
    - If not found:
        - Rank all the candidate by NSGA-II multi-objective losses
        - Return the top half of candidate CFs and new candidate CFs for this new generation

In [74]:
def generation(interaction, candidates, model, target, k, yloss_cache, crossover_p, mutation_p, budget):
    # print(len(candidates))
    pairs = generate_random_pairs(candidates, len(candidates)//2)
    t1 = time.time()
    for first, second in pairs:
        first, second = crossover(first, second, crossover_p)
        first = mutate_array(interaction, first, mutation_p)
        second = mutate_array(interaction, second, mutation_p)
        first = remove_duplicates(first)
        second = remove_duplicates(second)
        candidates.append(first)
        candidates.append(second)
    t2 = time.time()
    # print("Time taken for crossover and mutation: ", t2-t1)
    # print(len(candidates))
    t3 = time.time()
    losses = [compute_loss(interaction, arr, model, target, k, yloss_cache) for arr in candidates]
    # print(candidates)
    t4 = time.time()
    # print("Time taken for computing losses: ", t4-t3)
    budget -= len(candidates)
    # print(losses)
    solved = False
    solved_list = []
    for i in range(len(losses)):
        if losses[i][0] == 0:
            solved = True
            solved_list.append(candidates[i])
    if solved:
        return solved_list, solved, budget
    ranks, crowding_distances = fast_nondominated_sort(losses)
    # print(ranks)
    candidates_with_metrics = list(zip(candidates, ranks, crowding_distances))

    # Sort based on ranks (ascending) and then crowding distances (descending)
    candidates_with_metrics.sort(key=lambda x: (x[1], -x[2]))

    # Extract candidates after sorting
    sorted_candidates = [pair[0] for pair in candidates_with_metrics]

    # Extract the top third of candidates
    least_loss_arrays = sorted_candidates[:len(sorted_candidates)//2]

    return least_loss_arrays, solved, budget

In [75]:
def main(model, test_interaction, rank, sublists_info, top_k, crossover_p, mutation_p, budget):
    target = get_position_item(model, test_interaction, rank)
    new_gen = generate_random_sublists(test_interaction, sublists_info)
    solved = False
    yloss_cache = {}
    while solved is not True:
        new_gen, solved, budget = generation(test_interaction, new_gen, model, target, top_k, yloss_cache, crossover_p, mutation_p, budget)
        if budget <= 0:
            break
    return new_gen, budget, solved

# Brute Force Search
This section implements brute force search to find hard cases for testing and demostration. Here, hard cases means that it has to remove more than 3 items from the original interaction list to find at least one CF.

In [76]:
import numpy as np
import itertools

def subsets_of_array(arr, k):
    if not isinstance(arr, np.ndarray):
        raise ValueError("Input must be a numpy array")

    n = len(arr)
    subsets = []

    for size in range(n-1, n-k-1, -1):
        combinations = itertools.combinations(arr, size)
        for combo in combinations:
            subsets.append(np.array(combo))

    return subsets

In [77]:
def compute_loss_brute(new_cf, model, target_item, top_k, yloss_cache):
    cache_key = frozenset(new_cf)
    if cache_key in yloss_cache:
        yloss = yloss_cache[cache_key]
    else:
        new_prediction = model.predict(new_cf)
        new_prediction[new_cf] = -StaticVars.FLOAT_MAX
        new_rk_data = st.rankdata(-new_prediction, method='ordinal')

        top_k_index = np.where(new_rk_data == top_k)[0][0]
        yloss = compute_yloss(new_prediction[target_item], new_prediction[top_k_index])
        yloss_cache[cache_key] = yloss
    return yloss

In [78]:
def brute(candidates, model, target, top_k, yloss_cache):
    losses = [compute_loss_brute(arr, model, target, top_k, yloss_cache) for arr in candidates]
    solved = False
    for i in range(len(losses)):
        if losses[i] == 0:
            solved = True
    return solved

In [79]:
def brute_main(model, test_interaction, rank, top_k):
    target = get_position_item(model, test_interaction, rank)
    new_gen = subsets_of_array(test_interaction, 3)
    solved = False
    yloss_cache = {}
    solved = brute(new_gen, model, target, top_k, yloss_cache)
    return solved

The hard cases are stored in the file `final_list.txt` with first column the instance index and the second column the position of the target item.

In [80]:
# final_list = []
# for i in range(1000):
    # test_interaction = test.sequences[i].copy()
    # est_interaction = test_interaction[test_interaction != 0]
    # test_interaction.sort()
#     for j in range(1, 11):
#         solved = brute_main(pooling_model, test_interaction, j, 10)
#         if not solved:
#             print(i, j)
#             final_list.append((i, j))

# with open('final_list.txt', 'w') as file:
#     for item in final_list:
#         file.write(f"{item[0]}, {item[1]}\n")

# Testing hard case

The following piece of code loops through cossover_p and mutation_p from 0.1 to 0.5 with 0.1 incremental to tune the two parameters using the first 50 hard cases found in the previous section.

In [81]:
# test_crossover_p = [0.3, 0.4, 0.5]
# test_mutation_p = [0.1, 0.2, 0.3, 0.4, 0.5]
# result = []

# for crossover_p in test_crossover_p:
#     for mutation_p in test_mutation_p:
#         t0 = time.time()
#         solved_count = 0
#         with open("final_list.txt", 'r') as file:
#             counter = 0
#             for line in file:
#                 if counter == 50:
#                     break
#                 i, j = map(int, line.split(','))
#                 test_interaction = test.sequences[i].copy()
#                 test_interaction = test_interaction[test_interaction != 0]
#                 test_interaction.sort()
#                 length_interaction = len(test_interaction)
#                 if length_interaction <= 1:
#                     continue
#                 elif length_interaction < 5:
#                     sublists_info = {
#                         length_interaction - 1: length_interaction
#                     }
#                 else:
#                     sublists_info ={
#                         length_interaction - 1: length_interaction//2,
#                         length_interaction - 2: length_interaction * (length_interaction - 1) // 8,
#                         length_interaction - 3: length_interaction//3,
#                         length_interaction - 4: length_interaction//4
#                     }
#                 # print(sublists_info)
#                 output, budget, solved = main(pooling_model, test_interaction, j, sublists_info, 10, crossover_p, mutation_p, 1000)
#                 if solved:
#                     solved_count += 1
#                 print(budget)
#                 counter+=1
#         t1 = time.time()
#         result.append((crossover_p, mutation_p, t1-t0, solved_count))
#         print(result)
        


According to the test above, a acomparative optimal result comes from parameter with crossover_p = 0.3 and mutation_p = 0.2 

# Find CFs

The following function returns the longest array among a list of arrays. Here, it is used to find the most wanted CFs.

In [82]:
import numpy as np

def longest_array(arrays):
    if not arrays:
        return None  # Return None if the input list is empty
    
    max_length = 0
    longest_arr = None
    
    for arr in arrays:
        if arr.size > max_length:
            max_length = arr.size
            longest_arr = arr
            
    return longest_arr


By applying the most optimal crossover_p and mutation_p, the following function provides the same functionality as other strategies already implemented in `budget_strategies.ipynb` so it is comparable to them.

In [83]:
from tqdm import tqdm

def find_cf(test, model, targets, no_users, budget):
    cfs = dict.fromkeys(targets)
    
    # Progress bar for target loop
    for target in tqdm(targets, desc="target position loop", ncols=100):
        cfs[target] = []
        
        # Progress bar for users loop
        for i in trange(1, no_users + 1, desc='users loop', leave=False):
        # for i in tqdm(range(no_users), desc="users loop", ncols=100):
            # print(i)
            test_interaction = test.sequences[i].copy()
            test_interaction = test_interaction[test_interaction != 0]
            test_interaction.sort()
            length_interaction = len(test_interaction)
            if length_interaction <= 1:
                continue
            elif length_interaction < 5:
                sublists_info = {
                    length_interaction - 1: length_interaction
                }
            else:
                sublists_info ={
                    length_interaction - 1: length_interaction//2,
                    length_interaction - 2: length_interaction * (length_interaction - 1) // 10,
                    length_interaction - 3: length_interaction//3,
                    length_interaction - 4: length_interaction//4
                }
            # print(sublists_info)
            output, remain_budget, solved = main(model, test_interaction, target, sublists_info, 10, 0.3, 0.2, budget)
            # print(remain_budget)
            if solved:
                # print(output)
                cf = longest_array(output)
                cfs[target].extend(cf)
            else:
                cfs[target].append(None)
                
    return cfs


A test case of looping target positions of 1, 3, 5, 7 with 1000 user instances. Each case has a budget of 1000.

However, note that genetic algorithm might not have advantages on simple cases since it has heavy work loads in each generation. It works better for hard cases. Therefore, its performance is not as good as others.

Also, by turning how many initial candidate to generate in the begining might help in improving the performance as well. This is not yet implemented in this notebook.

In [84]:
find_cf(test, pooling_model, [1, 3, 5, 7], no_users=1000, budget=1000)

target position loop:   0%|                                                   | 0/4 [00:00<?, ?it/s]

users loop:   0%|          | 0/1000 [00:00<?, ?it/s]

target position loop:   0%|                                                   | 0/4 [01:23<?, ?it/s]


KeyboardInterrupt: 